In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
import holoviews as hv

iuc = pd.read_csv("iuc2(1)(1).csv",header=0)
up = [112,6,28,175,248,227,247,61,7,74]
down = [249,47,10,116,233,123,67,64,29,141]
# up_data = iuc[iuc["position"].isin(up)]
# down_data = iuc[iuc["position"].isin(down)]
all_data = iuc[iuc["position"].isin(up+down)]

def column_merge(a,b):
    return str([a,b])
# up_data['col_merge'] = up_data.apply(lambda row:column_merge(row['position'],row['amino_wt']),axis=1)
all_data['position_amino.wt'] = all_data.apply(lambda row:column_merge(row['position'],row['amino_wt']),axis=1)
all_data["pvalue"] = all_data["pvalue"].fillna(1)
all_data["-log10(pvalue)"] = -all_data['pvalue'].apply(np.log10)

all_data.loc[(all_data.pvalue <= 0.001),"symbol"] = "***"
all_data.loc[(all_data.pvalue <= 0.01),"symbol"] = "**"
all_data.loc[(all_data.pvalue <= 0.05),"symbol"] = "*"
all_data.loc[(all_data.pvalue > 0.05),"symbol"] = ""
all_data.loc[(all_data.amino_mut == all_data.amino_wt),"symbol"] = "○"

pn.extension()
x = pn.widgets.RadioButtonGroup(name='Function', options=['top10_up', 'top10_down',"top10_all"],width=300)
y = pn.widgets.RadioButtonGroup(name='Button Group', options=['-log10(pvalue)', 't3_vs_t0'],width=200)

@pn.depends(x,y,watch=True)
def pic(x,y):
    if x == "top10_up":
        data = all_data[all_data["position"].isin(up)]
        pheatmap = data.hvplot.heatmap(x='amino_mut', y='position_amino.wt', C=y,
                       cmap="Blues", xaxis='top', width=700,height=500).opts(toolbar=None, fontsize={'ticks': 10,'labels':15})
        plot = pheatmap * hv.Labels(pheatmap,vdims="symbol")
    elif x == "top10_down":
        data = all_data[all_data["position"].isin(down)]
        pheatmap = data.hvplot.heatmap(x='amino_mut', y='position_amino.wt', C=y,
                       cmap="Blues", xaxis='top', width=700,height=500).opts(toolbar=None, fontsize={'ticks': 10,'labels':15})
        plot = pheatmap * hv.Labels(pheatmap,vdims="symbol")
    else:
        data = all_data
        pheatmap = data.hvplot.heatmap(x='amino_mut', y='position_amino.wt', C=y,
                       cmap="Blues", xaxis='top', width=700,height=500).opts(toolbar=None, fontsize={'ticks': 10,'labels':15})
        plot = pheatmap * hv.Labels(pheatmap,vdims="symbol")
    return plot
pn.Row(pn.WidgetBox(x,y),pic)

display = pn.Row(pn.WidgetBox(x,y),pic)

pn.template.FastListTemplate(
    site="team5_homework", title="Heatmap of select position",
    main=["Heatmap of select position",display]).servable()